In [3]:
import tensorflow as tf
import os

#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

num_epochs = 8
batch_size = 64
learning_rate = 0.001
data_dir = './data/cats-vs-dogs/'
train_cats_dir = data_dir + 'train/cats/'
train_dogs_dir = data_dir + 'train/dogs/'
test_cats_dir = data_dir + 'valid/cats/'
test_dogs_dir = data_dir + 'valid/dogs/'
checkpoint_path = "training_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)                   # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)         # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [225, 225]) / 255.0
    return image_resized, label

class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.conv_7x7x64s2 = self.conv2d(7, 64, 2)
        self.pool3x3 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=2, padding='same')
        
        # inception1
        self.conv_1x1x64_in1 = self.conv2d(1, 64, 1)
        self.conv_1x1x32_in1 = self.conv2d(1, 32, 1)
        self.conv_3x3x64_in1 = self.conv2d(3, 64, 1)
        self.conv_1x1x16_in1 = self.conv2d(1, 16, 1)
        self.conv_5x5x32_in1 = self.conv2d(5, 32, 1)
        self.pool3x3s1_in1 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=1, padding='same')
        self.conv_1x1x32_in12 = self.conv2d(1, 64, 1)
        self.pool2x2_in1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        
        # inception2
        self.conv_1x1x128_in2 = self.conv2d(1, 128, 1)
        self.conv_1x1x64_in2 = self.conv2d(1, 64, 1)
        self.conv_3x3x128_in2 = self.conv2d(3, 128, 1)
        self.conv_1x1x32_in2 = self.conv2d(1, 32, 1)
        self.conv_5x5x64_in2 = self.conv2d(5, 64, 1)
        self.pool3x3s1_in2 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=1, padding='same')
        self.conv_1x1x64_in22 = self.conv2d(1, 64, 1)
        self.pool2x2_in2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        
        # inception3
        self.conv_1x1x128_in3 = self.conv2d(1, 128, 1)
        self.conv_1x1x64_in3 = self.conv2d(1, 64, 1)
        self.conv_3x3x128_in3 = self.conv2d(3, 128, 1)
        self.conv_1x1x32_in3 = self.conv2d(1, 32, 1)
        self.conv_5x5x64_in3 = self.conv2d(5, 64, 1)
        self.pool3x3s1_in3 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=1, padding='same')
        self.conv_1x1x64_in32 = self.conv2d(1, 64, 1)
        self.pool2x2_in3 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        
        # inception4
        self.conv_1x1x256_in4 = self.conv2d(1, 256, 1)
        self.conv_1x1x64_in4 = self.conv2d(1, 64, 1)
        self.conv_3x3x256_in4 = self.conv2d(3, 256, 1)
        self.conv_1x1x32_in4 = self.conv2d(1, 32, 1)
        self.conv_5x5x128_in4 = self.conv2d(5, 128, 1)
        self.pool3x3s1_in4 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=1, padding='same')
        self.conv_1x1x128_in4 = self.conv2d(1, 128, 1)
        
        self.concatenate = tf.keras.layers.Concatenate()

        self.avgpool7x7 = tf.keras.layers.AveragePooling2D(pool_size=[7, 7])
        self.flatten = tf.keras.layers.Reshape(target_shape=(1 * 1 * 768,))
        #self.avgpool = tf.keras.layers.GlobalAveragePooling2D  可以直接把每个特征图所有像素取一个均值点，
                                                                #代替上面的AveragePooling2D+Reshape
                                                                #output shape:(batch_size, channels)
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def conv2d(self, size, filters, stride):
        return tf.keras.layers.Conv2D(
            filters = filters,
            kernel_size = [size, size],
            strides = (stride, stride),
            padding = 'same',
            activation = tf.nn.relu
        )

    def inception1(self, inputs):
        x1 = self.conv_1x1x64_in1(inputs)
        x2 = self.conv_1x1x32_in1(inputs)
        x2 = self.conv_3x3x64_in1(x2)
        x3 = self.conv_1x1x16_in1(inputs)
        x3 = self.conv_5x5x32_in1(x3)
        x4 = self.pool3x3s1_in1(inputs)
        x4 = self.conv_1x1x32_in12(x4)
        output = self.concatenate([x1, x2, x3, x4])
        return output

    def inception2(self, inputs):
        x1 = self.conv_1x1x128_in2(inputs)
        x2 = self.conv_1x1x64_in2(inputs)
        x2 = self.conv_3x3x128_in2(x2)
        x3 = self.conv_1x1x32_in2(inputs)
        x3 = self.conv_5x5x64_in2(x3)
        x4 = self.pool3x3s1_in2(inputs)
        x4 = self.conv_1x1x64_in22(x4)
        output = self.concatenate([x1, x2, x3, x4])
        return output

    def inception3(self, inputs):
        x1 = self.conv_1x1x128_in3(inputs)
        x2 = self.conv_1x1x64_in3(inputs)
        x2 = self.conv_3x3x128_in3(x2)
        x3 = self.conv_1x1x32_in3(inputs)
        x3 = self.conv_5x5x64_in3(x3)
        x4 = self.pool3x3s1_in3(inputs)
        x4 = self.conv_1x1x64_in32(x4)
        output = self.concatenate([x1, x2, x3, x4])
        return output

    def inception4(self, inputs):
        x1 = self.conv_1x1x256_in4(inputs)
        x2 = self.conv_1x1x64_in4(inputs)
        x2 = self.conv_3x3x256_in4(x2)
        x3 = self.conv_1x1x32_in4(inputs)
        x3 = self.conv_5x5x128_in4(x3)
        x4 = self.pool3x3s1_in4(inputs)
        x4 = self.conv_1x1x128_in4(x4)
        output = self.concatenate([x1, x2, x3, x4])
        return output
    
    def call(self, inputs):
        x = self.conv_7x7x64s2(inputs)    # [112, 112, 64]
        x = self.pool3x3(x)    # [56, 56, 64]
        x = self.inception1(x) # [56, 56, 192]
        x = self.pool2x2_in1(x)    # [28, 28, 192]
        x = self.inception2(x) # [28, 28, 384]
        x = self.pool2x2_in2(x)    # [14, 14, 384]
        x = self.inception3(x) # [14, 14, 384]
        x = self.pool2x2_in3(x)    # [7, 7, 384]
        x = self.inception4(x) # [7, 7, 768]
        x = self.avgpool7x7(x)    # [1, 1, 768]
        x = self.flatten(x)   # [1*1*768]
        x = self.dense1(x)    # [256]
        x = self.dense2(x)    # [10]
        output = tf.nn.softmax(x)
        return output

def train_and_checkpoint(net, manager):
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch.")

    for example in toy_dataset():
        loss = train_step(net, example, opt)
        ckpt.step.assign_add(1)
        if int(ckpt.step) % 10 == 0:
            save_path = manager.save()
            print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
            print("loss {:1.2f}".format(loss.numpy()))

def train():
    # 构建训练数据集
    train_cat_filenames = tf.constant([train_cats_dir + filename for filename in os.listdir(train_cats_dir)])
    train_dog_filenames = tf.constant([train_dogs_dir + filename for filename in os.listdir(train_dogs_dir)])
    train_filenames = tf.concat([train_cat_filenames, train_dog_filenames], axis=-1)
    train_labels = tf.concat([
        tf.zeros(train_cat_filenames.shape, dtype=tf.int32),
        tf.ones(train_dog_filenames.shape, dtype=tf.int32)],
        axis=-1
    )
    
    train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
    train_dataset = train_dataset.map(
        map_func = _decode_and_resize,
        num_parallel_calls = tf.data.experimental.AUTOTUNE
    )# 取出前buffer_size个数据放入buffer，并从其中随机采样，采样后的数据用后续数据替换
    train_dataset = train_dataset.shuffle(buffer_size=23000)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    summary_writer = tf.summary.create_file_writer('./tensorboard')     # 训练过程可视化，参数为记录文件所保存的目录

    cp_callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)

    model = CNN()
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)  
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )
      
    model.fit(train_dataset, epochs=num_epochs, callbacks=[cp_callback_mc])
    model.summary()

def test():
    # 构建测试数据集
    test_cat_filenames = tf.constant([test_cats_dir + filename for filename in os.listdir(test_cats_dir)])
    test_dog_filenames = tf.constant([test_dogs_dir + filename for filename in os.listdir(test_dogs_dir)])
    test_filenames = tf.concat([test_cat_filenames, test_dog_filenames], axis=-1)
    test_labels = tf.concat([
        tf.zeros(test_cat_filenames.shape, dtype=tf.int32), 
        tf.ones(test_dog_filenames.shape, dtype=tf.int32)], 
        axis=-1)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames, test_labels))
    test_dataset = test_dataset.map(_decode_and_resize)
    test_dataset = test_dataset.batch(batch_size)

    model = CNN()
    # 加载权重
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )
        
    print(model.metrics_names)
    print(model.evaluate(test_dataset))

In [2]:
if __name__ ==  '__main__':
    train()

Epoch 1/8
360/360 [==============================] - 129s 358ms/step - loss: 0.0344 - sparse_categorical_accuracy: 0.9871
Epoch 2/8
360/360 [==============================] - 127s 353ms/step - loss: 0.0334 - sparse_categorical_accuracy: 0.9876
Epoch 3/8
360/360 [==============================] - 123s 341ms/step - loss: 0.0276 - sparse_categorical_accuracy: 0.9900
Epoch 4/8
360/360 [==============================] - 124s 343ms/step - loss: 0.0305 - sparse_categorical_accuracy: 0.9890
Epoch 5/8
360/360 [==============================] - 124s 344ms/step - loss: 0.0307 - sparse_categorical_accuracy: 0.9890
Epoch 6/8
360/360 [==============================] - 124s 344ms/step - loss: 0.0229 - sparse_categorical_accuracy: 0.9922
Epoch 7/8
360/360 [==============================] - 123s 342ms/step - loss: 0.0229 - sparse_categorical_accuracy: 0.9918
Epoch 8/8
360/360 [==============================] - 123s 341ms/step - loss: 0.0236 - sparse_categorical_accuracy: 0.9914
Model: "cnn"
___________

In [4]:
    test()

['loss']
32/32 [==============================] - 13s 406ms/step - loss: 0.4886 - sparse_categorical_accuracy: 0.9155
[0.48864244050309935, 0.9155]


In [ ]:
def get_one_image(dir):
    filename_list = os.listdir(dir)
    ind = np.random.randint(0, filename_list.len())
    return tf.constant([dir + filename_list[int]])

def evaluate_one_image():
    '''Test one image against the saved models and parameters
    '''

    # you need to change the directories to yours.
    # 数据集路径
    test_dir = 'F:/Downloads/fastai-datasets-cats-vs-dogs-2/test1/'
    test_filename = get_one_image(test_dir)      #调用get_one_image随机选取一幅图片并显示

    with tf.Graph().as_default():
        BATCH_SIZE = 1
        N_CLASSES = 2

        image = tf.cast(image_array, tf.float32)
        image = tf.image.per_image_standardization(image)
        image = tf.reshape(image, [1, 208, 208, 3])     #inference输入数据需要是4维数据，需要对image进行resize
        logit = model.inference(image, BATCH_SIZE, N_CLASSES)       
        logit = tf.nn.softmax(logit)    #inference的softmax层没有激活函数，这里增加激活函数

        #因为只有一副图，数据量小，所以用placeholder
        x = tf.placeholder(tf.float32, shape=[208, 208, 3])

        # you need to change the directories to yours.
        # 训练模型路径
        logs_train_dir = 'D:/Test/Cats_vs_Dogs/logs/train'

        saver = tf.train.Saver()

        with tf.Session() as sess:

            # 从指定路径下载模型
            print("Reading checkpoints...")
            ckpt = tf.train.get_checkpoint_state(logs_train_dir)
            if ckpt and ckpt.model_checkpoint_path:
                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                saver.restore(sess, ckpt.model_checkpoint_path)
                print('Loading success, global_step is %s' % global_step)
            else:
                print('No checkpoint file found')

            prediction = sess.run(logit, feed_dict={x: image_array})
            # 得到概率最大的索引
            max_index = np.argmax(prediction)
            if max_index==0:
                print('This is a cat with possibility %.6f' %prediction[:, 0])
            else:
                print('This is a dog with possibility %.6f' %prediction[:, 1])